In [4]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

import os


mb_size = 32

X_dim = 784

z_dim = 10

h_dim1 = 392

h_dim2 = 196

h_dim3 = 98



mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)


def xavier_init(size):

    in_dim = size[0]

    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)

    return tf.random_normal(shape=size, stddev=xavier_stddev)





X = tf.placeholder(tf.float32, shape=[None, 784])



D_W1 = tf.Variable(xavier_init([X_dim, h_dim1]))

D_b1 = tf.Variable(tf.zeros(shape=[h_dim1]))


D_W2 = tf.Variable(xavier_init([h_dim1, h_dim2]))

D_b2 = tf.Variable(tf.zeros(shape=[h_dim2]))


D_W3 = tf.Variable(xavier_init([h_dim2, h_dim3]))

D_b3 = tf.Variable(tf.zeros(shape=[h_dim3]))


D_W4 = tf.Variable(xavier_init([h_dim3, 1]))

D_b4 = tf.Variable(tf.zeros(shape=[1]))


theta_D = [D_W1, D_W2, D_W3, D_W4, D_b1, D_b2, D_b3, D_b4]





z = tf.placeholder(tf.float32, shape=[None, z_dim])


G_W1 = tf.Variable(xavier_init([z_dim, h_dim3]))

G_b1 = tf.Variable(tf.zeros(shape=[h_dim3]))


G_W2 = tf.Variable(xavier_init([h_dim3, h_dim2]))

G_b2 = tf.Variable(tf.zeros(shape=[h_dim2]))


G_W3 = tf.Variable(xavier_init([h_dim2, h_dim1]))

G_b3 = tf.Variable(tf.zeros(shape=[h_dim1]))


G_W4 = tf.Variable(xavier_init([h_dim1, X_dim]))

G_b4 = tf.Variable(tf.zeros(shape=[X_dim]))



theta_G = [G_W1, G_W2, G_W3, G_W4, G_b1, G_b2, G_b3, G_b4]





def sample_Z(m, n):

    return np.random.uniform(-1., 1., size=[m, n])





def generator(z):

    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
    
    G_h3 = tf.nn.relu(tf.matmul(G_h2, G_W3) + G_b3)

    G_log_prob = tf.matmul(G_h3, G_W4) + G_b4

    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob





def discriminator(x):

    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    
    D_h3 = tf.nn.relu(tf.matmul(D_h2, D_W3) + D_b3)

    D_logit = tf.matmul(D_h3, D_W4) + D_b4

    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit





def plot(samples):

    fig = plt.figure(figsize=(4, 4))

    gs = gridspec.GridSpec(4, 4)

    gs.update(wspace=0.05, hspace=0.05)



    for i, sample in enumerate(samples):

        ax = plt.subplot(gs[i])

        plt.axis('off')

        ax.set_xticklabels([])

        ax.set_yticklabels([])

        ax.set_aspect('equal')

        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')



    return fig





G_sample = generator(z)

D_real, D_logit_real = discriminator(X)

D_fake, D_logit_fake = discriminator(G_sample)



# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))

# G_loss = -tf.reduce_mean(tf.log(D_fake))



# Alternative losses:

# -------------------

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))

D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))

D_loss = D_loss_real + D_loss_fake

G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))



D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)

G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)




#mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)



sess = tf.Session()

sess.run(tf.global_variables_initializer())



if not os.path.exists('out/'):

    os.makedirs('out/')



i = 0



Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz


In [10]:

for it in range(300000):

    if it % 1000 == 0:

        samples = sess.run(G_sample, feed_dict={z: sample_Z(16, z_dim)})



        fig = plot(samples)

        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')

        i += 1

        plt.close(fig)



    X_mb, _ = mnist.train.next_batch(mb_size)



    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, z: sample_Z(mb_size, z_dim)})

    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={z: sample_Z(mb_size, z_dim)})



    if it % 1000 == 0:

        print('Iter: {}'.format(it))

        print('D loss: {:.4}'. format(D_loss_curr))

        print('G_loss: {:.4}'.format(G_loss_curr))

        print()
        

        
print(sess.run(theta_G))
np.save('gan.npy', sess.run(theta_G))

Iter: 0
D loss: 1.416
G_loss: 2.737

Iter: 1000
D loss: 5.539e-06
G_loss: 12.21

Iter: 2000
D loss: 5.332e-07
G_loss: 14.47

Iter: 3000
D loss: 5.743e-07
G_loss: 14.38

Iter: 4000
D loss: 1.558e-06
G_loss: 13.44

Iter: 5000
D loss: 2.393e-08
G_loss: 17.55

Iter: 6000
D loss: 1.006e-22
G_loss: 59.56

Iter: 7000
D loss: 1.253e-14
G_loss: 32.01

Iter: 8000
D loss: 1.253e-14
G_loss: 32.01

Iter: 9000
D loss: 1.253e-14
G_loss: 32.01

Iter: 10000
D loss: 4.476e-11
G_loss: 23.83

Iter: 11000
D loss: 1.108e-10
G_loss: 22.92

Iter: 12000
D loss: 3.965e-11
G_loss: 23.95

Iter: 13000
D loss: 2.36e-11
G_loss: 24.47

Iter: 14000
D loss: 1.394e-11
G_loss: 25.0

Iter: 15000
D loss: 1.029e-09
G_loss: 20.75

Iter: 16000
D loss: 2.114e-11
G_loss: 24.58

Iter: 17000
D loss: 7.582e-12
G_loss: 25.61

Iter: 18000
D loss: 4.263e-12
G_loss: 26.18

Iter: 19000
D loss: 4.967e-12
G_loss: 26.57

Iter: 20000
D loss: 2.067e-12
G_loss: 26.91

Iter: 21000
D loss: 1.551e-12
G_loss: 27.19

Iter: 22000
D loss: 1.309e-12

Iter: 182000
D loss: 3.015e-14
G_loss: 31.13

Iter: 183000
D loss: 2.994e-14
G_loss: 31.14

Iter: 184000
D loss: 2.576e-14
G_loss: 31.29

Iter: 185000
D loss: 3.274e-14
G_loss: 31.05

Iter: 186000
D loss: 2.239e-14
G_loss: 31.43

Iter: 187000
D loss: 1.963e-14
G_loss: 31.56

Iter: 188000
D loss: 2.525e-14
G_loss: 31.31

Iter: 189000
D loss: 3e-14
G_loss: 31.14

Iter: 190000
D loss: 3.27e-14
G_loss: 31.05

Iter: 191000
D loss: 2.104e-14
G_loss: 31.49

Iter: 192000
D loss: 1.671e-14
G_loss: 31.72

Iter: 193000
D loss: 3.003e-14
G_loss: 31.14

Iter: 194000
D loss: 2.096e-14
G_loss: 31.5

Iter: 195000
D loss: 2.024e-14
G_loss: 31.53

Iter: 196000
D loss: 2.041e-14
G_loss: 31.52

Iter: 197000
D loss: 2.136e-14
G_loss: 31.48

Iter: 198000
D loss: 1.853e-14
G_loss: 31.62

Iter: 199000
D loss: 1.861e-14
G_loss: 31.62

Iter: 200000
D loss: 1.872e-14
G_loss: 31.61

Iter: 201000
D loss: 1.629e-14
G_loss: 31.75

Iter: 202000
D loss: 1.814e-14
G_loss: 31.64

Iter: 203000
D loss: 1.477e-14
G_loss: 3